In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set()

In [2]:
data_pre=pd.read_csv("preprocessing.csv")
data_pre.head(10)

,Gender,Occupation,Channel_Code,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Age_Group,Vintage_Group
0,0,1,2,1,0.099205,0,0,78,24
1,0,2,0,1,0.054321,0,0,34,24
2,0,3,2,1,0.141660,1,0,56,24
3,1,2,0,1,0.043525,0,0,34,2
4,0,2,0,1,0.083823,0,0,34,24
5,1,3,0,1,0.050659,1,0,56,24
6,1,1,2,2,0.100275,1,1,67,2
7,0,3,2,1,0.041034,1,0,45,2
8,0,3,1,1,0.121331,0,0,45,24
9,0,3,1,0,0.192954,0,0,56,46


##### Pre Processing

In [3]:
#droping target-variable for the pre
X=data_pre.drop('Is_Lead', axis=1)
Y=data_pre['Is_Lead']

# Splitting to Training and Test
from sklearn.model_selection import train_test_split
train_x, test_x , train_y, test_y = train_test_split(X, Y, train_size=0.55, test_size = 0.45)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

((135148, 8), (110577, 8), (135148,), (110577,))

## Training Model and data validation

#### Logistic Regression

In [4]:
# import 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

model=LogisticRegression()
model.fit(train_x,train_y)
pred_prob=model.predict_proba(test_x)
auc_score=roc_auc_score(test_y, pred_prob[:,1])
print("Logistic Regression = " + str(auc_score))

Logistic Regression = 0.7249274460103249


In [5]:
model.coef_

array([[ 0.11704892,  0.0224854 ,  0.47832673,  0.70136698, -0.18585973,
        -0.06044984,  0.0069836 ,  0.01218743]])

In [6]:
coef=model.coef_.reshape(-1)
Coefficient= pd.DataFrame({'Variable':train_x.columns, 'coef':coef})
Coefficient.sort_values(by='coef')

,Variable,coef
4,Avg_Account_Balance,-0.185860
5,Is_Active,-0.060450
6,Age_Group,0.006984
7,Vintage_Group,0.012187
1,Occupation,0.022485
0,Gender,0.117049
2,Channel_Code,0.478327
3,Credit_Product,0.701367


In [7]:
data_fmt=data_pre[['Credit_Product','Channel_Code','Gender','Avg_Account_Balance', 'Is_Active', 'Is_Lead']]
#droping target-variable for the pre
X=data_fmt.drop('Is_Lead', axis=1)
Y=data_fmt['Is_Lead']

# Splitting to Training and Test
from sklearn.model_selection import train_test_split
train_x, test_x , train_y, test_y = train_test_split(X, Y, train_size=0.55, test_size = 0.45)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

((135148, 5), (110577, 5), (135148,), (110577,))

In [8]:
model1=LogisticRegression()
model1.fit(train_x,train_y)
pred_prob=model1.predict_proba(test_x)
auc_score=roc_auc_score(test_y, pred_prob[:,1])
print("Logistic Regression = " + str(auc_score))

Logistic Regression = 0.7019513085866553


## Test Prediction

In [9]:
test=pd.read_csv("test.csv")
test.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No


In [10]:
## Age
test=test.assign(Age_Group = test['Age'].apply(lambda x : 23 if (x>=20 and x<30)
                                               else (34 if (x>=30 and x<40)
                                                     else (45 if (x>=40 and x<50)
                                                           else (56 if (x>=50 and x<60)
                                                                 else (67 if (x>=60 and x<70)
                                                                       else (78 if (x>=70 and x<80)
                                                                             else 80)))))))
test.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Age_Group
0,VBENBARO,Male,29,RG254,Other,X1,25,Yes,742366,No,23
1,CCMEWNKY,Male,43,RG268,Other,X2,49,NaN,925537,No,45
2,VK3KGA9M,Male,31,RG270,Salaried,X1,14,No,215949,No,34
3,TT8RPZVC,Male,29,RG272,Other,X1,33,No,868070,No,23
4,SHQZEYTZ,Female,29,RG270,Other,X1,19,No,657087,No,23


In [11]:
test['Credit_Product']=test['Credit_Product'].fillna('0')

## Vintage
test=test.assign(Vintage_Group = test['Vintage'].apply(lambda x : 2 if (x>=0 and x<24)
                                               else (24 if (x>=24 and x<48)
                                                     else (46 if (x>=48 and x<72)
                                                           else (68 if (x>=72 and x<96)
                                                                 else (81 if (x>=96 and x<120)
                                                                       else 99))))))

test=test.drop(['Age','Vintage'], axis=1)
test['Gender'].replace({'Male':1, 'Female':0}, inplace=True)
test['Occupation'].replace({'Entrepreneur':0, 'Other':1, 'Salaried':2, 'Self_Employed':3}, inplace=True)
test['Channel_Code'].replace({'X1':0, 'X2':1, 'X3':2, 'X4':3}, inplace=True)
test['Is_Active'].replace({'Yes':1, 'No':0}, inplace=True)
test['Credit_Product'].replace({'Yes':0,'No':1,'0':2},inplace=True)
test.head()

,ID,Gender,Region_Code,Occupation,Channel_Code,Credit_Product,Avg_Account_Balance,Is_Active,Age_Group,Vintage_Group
0,VBENBARO,1,RG254,1,0,0,742366,0,23,24
1,CCMEWNKY,1,RG268,1,1,2,925537,0,45,46
2,VK3KGA9M,1,RG270,2,0,1,215949,0,34,2
3,TT8RPZVC,1,RG272,1,0,1,868070,0,23,24
4,SHQZEYTZ,0,RG270,1,0,1,657087,0,23,2


In [12]:
test_tst=test.drop(['ID','Region_Code'], axis=1)


#scaling Avg_Account_Balance with Standard-Deviation
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
test_tst['Avg_Account_Balance'] = ss.fit_transform(test_tst[['Avg_Account_Balance']])

In [13]:
predicted_val=model.predict(test_tst)
Final=pd.DataFrame({'ID':test['ID'],'Is_Lead':predicted_val})
Final.to_csv('LR-Final.csv',index=False)

In [14]:
test_tst1=test_tst[['Credit_Product','Channel_Code','Gender','Avg_Account_Balance','Is_Active']]
predicted_val1=model1.predict(test_tst1)
Final1=pd.DataFrame({'ID':test['ID'],'Is_Lead':predicted_val1})
Final1.to_csv('LR-Final_Coeff.csv',index=False)